In [1]:
from pyspark.sql import SparkSession
import getpass 
username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/itv012740/warehouse"). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/user/itv012740/lendingclubproject/raw/loans_repayments_csv")

In [3]:
loans_repay_raw_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
69642551,3246.03,990.53,0.0,4965.02,326.35,Feb-2017,null
69336966,12000.0,1080.5,0.0,13080.5,9501.81,Nov-2016,null
69226186,28000.0,3049.36,0.0,31049.361806989902,9306.81,Mar-2018,null
69084024,16500.0,900.32,0.0,17400.32,14085.82,Aug-2016,null
69422635,21000.0,4452.42,0.0,25452.4171310098,12705.78,Jun-2018,null
69542436,1220.15,1252.23,0.0,4533.19,497.94,Jul-2016,null
69492501,29000.0,5405.32,47.8,34453.12,771.08,Mar-2019,Apr-2019
69522527,33000.0,2092.69,0.0,35092.69,65.34,Jul-2016,null
68781653,18000.0,4167.34,0.0,22167.3434977632,13021.32,Jan-2018,null
69371334,16000.0,1140.7,0.0,17140.7005770789,13910.33,Dec-2016,null


In [4]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_rec_prncp: string (nullable = true)
 |-- total_rec_int: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- total_pymnt: string (nullable = true)
 |-- last_pymnt_amnt: string (nullable = true)
 |-- last_pymnt_d: string (nullable = true)
 |-- next_pymnt_d: string (nullable = true)



In [5]:
loans_repay_schema = 'loan_id string, total_principal_received float, total_interest_received float, total_late_fee_received float, total_payment_received float, last_payment_amount float, last_payment_date string, next_payment_date string'

In [6]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loans_repay_schema) \
.load("/user/itv012740/lendingclubproject/raw/loans_repayments_csv")

In [7]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)



In [8]:
from pyspark.sql.functions import current_timestamp

In [9]:
loans_repay_df_ingestd = loans_repay_raw_df.withColumn("ingest_date", current_timestamp())

In [10]:
loans_repay_df_ingestd

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
69642551,3246.03,990.53,0.0,4965.02,326.35,Feb-2017,null,2024-12-15 06:05:...
69336966,12000.0,1080.5,0.0,13080.5,9501.81,Nov-2016,null,2024-12-15 06:05:...
69226186,28000.0,3049.36,0.0,31049.361,9306.81,Mar-2018,null,2024-12-15 06:05:...
69084024,16500.0,900.32,0.0,17400.32,14085.82,Aug-2016,null,2024-12-15 06:05:...
69422635,21000.0,4452.42,0.0,25452.418,12705.78,Jun-2018,null,2024-12-15 06:05:...
69542436,1220.15,1252.23,0.0,4533.19,497.94,Jul-2016,null,2024-12-15 06:05:...
69492501,29000.0,5405.32,47.8,34453.12,771.08,Mar-2019,Apr-2019,2024-12-15 06:05:...
69522527,33000.0,2092.69,0.0,35092.69,65.34,Jul-2016,null,2024-12-15 06:05:...
68781653,18000.0,4167.34,0.0,22167.344,13021.32,Jan-2018,null,2024-12-15 06:05:...
69371334,16000.0,1140.7,0.0,17140.701,13910.33,Dec-2016,null,2024-12-15 06:05:...


In [11]:
loans_repay_df_ingestd.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [12]:
loans_repay_df_ingestd.count()

2260701

In [13]:
loans_repay_df_ingestd.createOrReplaceTempView("loan_repayments")

In [14]:
spark.sql("select count(*) from loan_repayments where total_principal_received is null")

count(1)
69


In [15]:
columns_to_check = ["total_principal_received", "total_interest_received", "total_late_fee_received", "total_payment_received", "last_payment_amount"]

In [16]:
loans_repay_filtered_df = loans_repay_df_ingestd.na.drop(subset=columns_to_check)

In [17]:
loans_repay_filtered_df.count()

2260498

In [18]:
loans_repay_filtered_df.createOrReplaceTempView("loan_repayments")

In [19]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0")

count(1)
995


In [20]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0 and total_principal_received != 0.0")

count(1)
46


In [21]:
spark.sql("select * from loan_repayments where total_payment_received = 0.0 and total_principal_received != 0.0")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
485818,14640.096,13388.84,13000.0,0.0,0.0,0.0,Mar-2013,2024-12-15 06:05:...
485471,29620.818,29134.64,25000.0,0.0,0.0,0.0,Mar-2013,2024-12-15 06:05:...
482256,8735.611,7479.87,8000.0,0.0,0.0,0.0,Feb-2011,2024-12-15 06:05:...
478160,410.0,407.36,0.0,0.0,143.1,410.0,null,2024-12-15 06:05:...
476557,28865.18,24164.67,5692.31,0.0,6972.59,19916.78,Dec-2010,2024-12-15 06:05:...
472516,25951.482,24731.76,25000.0,0.0,0.0,0.0,May-2010,2024-12-15 06:05:...
472197,12048.13,12018.01,10000.0,0.0,0.0,0.0,Jan-2013,2024-12-15 06:05:...
467364,29216.791,29066.19,24250.0,0.0,0.0,0.0,Dec-2012,2024-12-15 06:05:...
399499,26557.729,26336.41,24000.0,0.0,0.0,0.0,Dec-2010,2024-12-15 06:05:...
451482,7587.5513,7587.55,7000.0,0.0,0.0,0.0,Jan-2011,2024-12-15 06:05:...


In [22]:
from pyspark.sql.functions import when, col

In [23]:
loans_payments_fixed_df = loans_repay_filtered_df.withColumn(
   "total_payment_received",
    when(
        (col("total_principal_received") != 0.0) &
        (col("total_payment_received") == 0.0),
        col("total_principal_received") + col("total_interest_received") + col("total_late_fee_received")
    ).otherwise(col("total_payment_received"))
)

In [24]:
loans_payments_fixed_df

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
69642551,3246.03,990.53,0.0,4965.02,326.35,Feb-2017,null,2024-12-15 06:05:...
69336966,12000.0,1080.5,0.0,13080.5,9501.81,Nov-2016,null,2024-12-15 06:05:...
69226186,28000.0,3049.36,0.0,31049.361,9306.81,Mar-2018,null,2024-12-15 06:05:...
69084024,16500.0,900.32,0.0,17400.32,14085.82,Aug-2016,null,2024-12-15 06:05:...
69422635,21000.0,4452.42,0.0,25452.418,12705.78,Jun-2018,null,2024-12-15 06:05:...
69542436,1220.15,1252.23,0.0,4533.19,497.94,Jul-2016,null,2024-12-15 06:05:...
69492501,29000.0,5405.32,47.8,34453.12,771.08,Mar-2019,Apr-2019,2024-12-15 06:05:...
69522527,33000.0,2092.69,0.0,35092.69,65.34,Jul-2016,null,2024-12-15 06:05:...
68781653,18000.0,4167.34,0.0,22167.344,13021.32,Jan-2018,null,2024-12-15 06:05:...
69371334,16000.0,1140.7,0.0,17140.701,13910.33,Dec-2016,null,2024-12-15 06:05:...


In [25]:
loans_payments_fixed_df.filter("loan_id == '1064185'")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
1064185,11600.98,11600.98,10000.0,33201.96,0.0,0.0,Dec-2014,2024-12-15 06:06:...


In [26]:
loans_payments_fixed2_df = loans_payments_fixed_df.filter("total_payment_received != 0.0")

In [27]:
loans_payments_fixed2_df.filter("last_payment_date = 0.0").count()

48

In [28]:
loans_payments_fixed2_df.filter("next_payment_date = 0.0").count()

24

In [29]:
loans_payments_fixed2_df.filter("last_payment_date is null").count()

1477

In [30]:
loans_payments_fixed2_df.filter("next_payment_date is null").count()

1344240

In [31]:
loans_payments_ldate_fixed_df = loans_payments_fixed2_df.withColumn(
  "last_payment_date",
   when(
       (col("last_payment_date") == 0.0),
       None
       ).otherwise(col("last_payment_date"))
)

In [32]:
loans_payments_ndate_fixed_df = loans_payments_ldate_fixed_df.withColumn(
  "last_payment_date",
   when(
       (col("next_payment_date") == 0.0),
       None
       ).otherwise(col("next_payment_date"))
)

In [33]:
loans_payments_ndate_fixed_df.filter("last_payment_date = 0.0").count()

0

In [34]:
loans_payments_ndate_fixed_df.filter("next_payment_date = 0.0").count()

24

In [36]:
loans_payments_ndate_fixed_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv012740/lendingclubproject/raw/cleaned/loans_repayments_parquet") \
.save()

In [37]:
loans_payments_ndate_fixed_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv012740/lendingclubproject/raw/cleaned/loans_repayments_csv") \
.save()